# Working on Coefficients

### Coefficients concatenation

In [17]:
import pandas as pd

In [18]:
stations_event = ["PORTE DE SAINT-CLOUD", "PORTE D'AUTEUIL", "BOULEVARD VICTOR", "STADE DE FRANCE-SAINT-DENIS", "LA PLAINE-STADE DE FRANCE", "SAINT-DENIS-PORTE DE PARIS"]

In [19]:
df_coeff = pd.read_csv(f"98-Forecast/coeff/coeff_{stations_event[0]}.csv")
df_coeff

,regressor,regressor_mode,center,coef_lower,coef,coef_upper,station
0,EVENT_RG,additive,0.000000,1.285228,1.285228,1.285228,PORTE DE SAINT-CLOUD
1,EVENT_PDP,additive,0.000000,-16.139999,-16.139999,-16.139999,PORTE DE SAINT-CLOUD
2,EVENT_SDF,additive,0.000000,0.000000,0.000000,0.000000,PORTE DE SAINT-CLOUD
3,EVENT_RG_LOG,additive,0.422003,-0.113329,-0.113329,-0.113329,PORTE DE SAINT-CLOUD
4,EVENT_PDP_LOG,additive,0.803951,1.520497,1.520497,1.520497,PORTE DE SAINT-CLOUD
5,EVENT_SDF_LOG,additive,0.000000,0.000000,0.000000,0.000000,PORTE DE SAINT-CLOUD


In [20]:
for station in stations_event[1:]:
    df_station = pd.read_csv(f"98-Forecast/coeff/coeff_{station}.csv")
    df_coeff = pd.concat([df_coeff, df_station], axis=0)
df_coeff.head(10)

,regressor,regressor_mode,center,coef_lower,coef,coef_upper,station
0,EVENT_RG,additive,0.000000,1.285228,1.285228,1.285228,PORTE DE SAINT-CLOUD
1,EVENT_PDP,additive,0.000000,-16.139999,-16.139999,-16.139999,PORTE DE SAINT-CLOUD
2,EVENT_SDF,additive,0.000000,0.000000,0.000000,0.000000,PORTE DE SAINT-CLOUD
3,EVENT_RG_LOG,additive,0.422003,-0.113329,-0.113329,-0.113329,PORTE DE SAINT-CLOUD
4,EVENT_PDP_LOG,additive,0.803951,1.520497,1.520497,1.520497,PORTE DE SAINT-CLOUD
5,EVENT_SDF_LOG,additive,0.000000,0.000000,0.000000,0.000000,PORTE DE SAINT-CLOUD
0,EVENT_RG,additive,0.000000,-11.520813,-11.520813,-11.520813,PORTE D'AUTEUIL
1,EVENT_PDP,additive,0.000000,-11.373046,-11.373046,-11.373046,PORTE D'AUTEUIL
2,EVENT_SDF,additive,0.000000,0.000000,0.000000,0.000000,PORTE D'AUTEUIL
3,EVENT_RG_LOG,additive,0.421423,1.169022,1.169022,1.169022,PORTE D'AUTEUIL


In [21]:
df_coeff.to_csv('98-Forecast/coeff/coeff.csv', index=False)

### Coefficient calculation

In [22]:
stations_ssevent = ["FRANKLIN-D.ROOSEVELT", "INVALIDES", "PALAIS ROYAL-MUSEE DU LOUVRE", "MADELEINE", "OPERA", "AUBER", 
                    "MUSEE D'ORSAY", "CHAMP DE MARS-TOUR EIFFEL", "LA MOTTE-PICQUET-GRENELLE", "SEGUR", "PONT DE L'ALMA", 
                    "BIR-HAKEIM (GRENELLE)", "ALMA-MARCEAU", "PORTE DE VERSAILLES", "BALARD", "PORTE DE VANVES", "PORTE DE LA CHAPELLE", 
                    "ROSA PARKS", "BERCY", "GARE DE LYON"]


In [23]:
df_capacite = pd.read_csv('99-Data_Clean/geoloc_jo24.csv')
df_capacite[df_capacite['type'] == 'site'].sort_values(by='capacite')

,lieu,type,lat,lng,capacite
36,ARENA LA CHAPELLE,site,48.899483,2.361803,7000.0
28,GRAND PALAIS,site,48.866041,2.312406,8000.0
29,ARENA INVALIDES,site,48.862286,2.313222,8000.0
32,ARENA CHAMP DE MARS,site,48.854536,2.300413,8356.0
33,STADE TOUR EIFFEL,site,48.857232,2.296084,12860.0
31,STADE YVES DU MANOIR,site,48.930873,2.249836,15000.0
37,ARENA BERCY,site,48.838649,2.378496,15000.0
35,ARENA PARIS SUD,site,48.832068,2.284126,26450.0
30,STADE DE LA CONCORDE,site,48.865589,2.321205,30000.0
34,ROLAND-GARROS,site,48.847320,2.245501,34000.0


None of sites without events known are bigger than ROLAND-GARROS, the one with the lowest capacity of sites with events known.
We will apply coefficient of ROLAND-GARROS to other sites without events known.

In [24]:
stations_RG = ["PORTE DE SAINT-CLOUD", "PORTE D'AUTEUIL"]

In [25]:
porte_stcloud_cap = df_capacite[(df_capacite['lieu'] == stations_RG[0])]['capacite']
porte_stcloud_cap

5    17029.0
Name: capacite, dtype: float64

In [26]:
porte_auteuil_cap = df_capacite[(df_capacite['lieu'] == stations_RG[1])]['capacite']
porte_auteuil_cap

4    1968.0
Name: capacite, dtype: float64

In [27]:
diff = (int(porte_stcloud_cap.values) - int(porte_auteuil_cap.values))/2
diff

7530.5

In [28]:
df_coeff_stations_ssevent = df_capacite[df_capacite['type'] == 'station'].sort_values(by='capacite')
df_coeff_stations_ssevent = df_coeff_stations_ssevent[df_coeff_stations_ssevent['lieu'].isin(stations_ssevent)]
df_coeff_stations_ssevent = df_coeff_stations_ssevent.reset_index().drop('index', axis=1)
df_coeff_stations_ssevent['site model'] = 'ROLAND-GARROS'
df_coeff_stations_ssevent['station model'] = 0
df_coeff_stations_ssevent.head()

,lieu,type,lat,lng,capacite,site model,station model
0,MUSEE D'ORSAY,station,48.860570,2.325451,4276.0,ROLAND-GARROS,0
1,SEGUR,station,48.847104,2.307204,5347.0,ROLAND-GARROS,0
2,PONT DE L'ALMA,station,48.862514,2.300916,5553.0,ROLAND-GARROS,0
3,CHAMP DE MARS-TOUR EIFFEL,station,48.855093,2.288920,7457.0,ROLAND-GARROS,0
4,PORTE DE LA CHAPELLE,station,48.898125,2.358791,8737.0,ROLAND-GARROS,0


In [29]:
for i in range(len(df_coeff_stations_ssevent['lieu'])):
    if diff > int(df_coeff_stations_ssevent[(df_coeff_stations_ssevent['lieu'] == df_coeff_stations_ssevent['lieu'][i])]['capacite'].values):
        df_coeff_stations_ssevent['station model'][i] = stations_RG[1]
    else:
        df_coeff_stations_ssevent['station model'][i] = stations_RG[0]
df_coeff_stations_ssevent = df_coeff_stations_ssevent[['lieu', 'site model', 'station model']]
df_coeff_stations_ssevent.head()

/var/folders/5n/p64q8ht13qzd7qbp48cbvk880000gn/T/ipykernel_77748/654810896.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_coeff_stations_ssevent['station model'][i] = stations_RG[1]


,lieu,site model,station model
0,MUSEE D'ORSAY,ROLAND-GARROS,PORTE D'AUTEUIL
1,SEGUR,ROLAND-GARROS,PORTE D'AUTEUIL
2,PONT DE L'ALMA,ROLAND-GARROS,PORTE D'AUTEUIL
3,CHAMP DE MARS-TOUR EIFFEL,ROLAND-GARROS,PORTE D'AUTEUIL
4,PORTE DE LA CHAPELLE,ROLAND-GARROS,PORTE DE SAINT-CLOUD


In [30]:
RG_AUTEUIL = float(df_coeff[(df_coeff['station'] == stations_RG[1]) & (df_coeff['regressor'] == "EVENT_RG")]['coef'].values)
RG_STCLOUD = float(df_coeff[(df_coeff['station'] == stations_RG[0]) & (df_coeff['regressor'] == "EVENT_RG")]['coef'].values)
RG_LOG_AUTEUIL = float(df_coeff[(df_coeff['station'] == stations_RG[1]) & (df_coeff['regressor'] == "EVENT_RG_LOG")]['coef'].values)
RG_LOG_STCLOUD = float(df_coeff[(df_coeff['station'] == stations_RG[0]) & (df_coeff['regressor'] == "EVENT_RG_LOG")]['coef'].values)

In [31]:
df_coeff_stations_ssevent['EVENT'] = df_coeff_stations_ssevent['station model'].apply(lambda x: RG_AUTEUIL if x == stations_RG[1] else RG_STCLOUD)
df_coeff_stations_ssevent['EVENT_LOG'] = df_coeff_stations_ssevent['station model'].apply(lambda x: RG_LOG_AUTEUIL if x == stations_RG[1] else RG_LOG_STCLOUD)
df_coeff_stations_ssevent

,lieu,site model,station model,EVENT,EVENT_LOG
0,MUSEE D'ORSAY,ROLAND-GARROS,PORTE D'AUTEUIL,-11.520813,1.169022
1,SEGUR,ROLAND-GARROS,PORTE D'AUTEUIL,-11.520813,1.169022
2,PONT DE L'ALMA,ROLAND-GARROS,PORTE D'AUTEUIL,-11.520813,1.169022
3,CHAMP DE MARS-TOUR EIFFEL,ROLAND-GARROS,PORTE D'AUTEUIL,-11.520813,1.169022
4,PORTE DE LA CHAPELLE,ROLAND-GARROS,PORTE DE SAINT-CLOUD,1.285228,-0.113329
5,ALMA-MARCEAU,ROLAND-GARROS,PORTE DE SAINT-CLOUD,1.285228,-0.113329
6,BIR-HAKEIM (GRENELLE),ROLAND-GARROS,PORTE DE SAINT-CLOUD,1.285228,-0.113329
7,BERCY,ROLAND-GARROS,PORTE DE SAINT-CLOUD,1.285228,-0.113329
8,INVALIDES,ROLAND-GARROS,PORTE DE SAINT-CLOUD,1.285228,-0.113329
9,PORTE DE VANVES,ROLAND-GARROS,PORTE DE SAINT-CLOUD,1.285228,-0.113329


In [32]:
df_coeff_stations_ssevent.to_csv("99-Data_Clean/coeff_stations_ssevent.csv", index=False)